# Imports

In [84]:
import numpy as np

### Question 1

#### Task 1

In [85]:
DataMatrix = [[5, 5, 4, 1, 1], [4, 5, 5, 0, 1], [5, 4, 5, 1, 0], [4, 5, 4, 1, 1], [0, 1, 2, 5, 5], [1, 2, 1, 4, 5]]
DataMatrix = np.array(DataMatrix)

U, sigma, VT = np.linalg.svd(DataMatrix)
Sigma = np.zeros((6, 5))

for i in range(5):
    Sigma[i][i] = sigma[i]

print(sigma)

[16.75945308  8.77892061  1.71703139  1.3553879   0.51576392]


#### Task 2

$\textbf{U}$ represents the latent factors associated with users, while $\textbf{V}^T$ represents the latent factors associated with movies. Latent factors reflect the general preference of the users while $\mathbf{\Sigma}$ consists of the weights of these latent preference factors. 

#### Task 3

In [86]:
k = 4   # reduction factor

U_reduced, sigma_reduced, Vt_reduced = U[:, :k], sigma[:k], VT[:, :]
Sigma_reduced = np.zeros((k, 5))
for i in range(k):
    Sigma_reduced[i][i] = sigma_reduced[i]

A = U_reduced @ Sigma_reduced @ Vt_reduced

# print(U_reduced.shape, Sigma_reduced.shape, Vt_reduced.shape)
# print()

print(A)
print()
print(DataMatrix)

[[ 4.98558332  5.0171235   3.99665013  1.0165784   0.9821824 ]
 [ 3.91364802  5.10256509  4.97993518  0.09930004  0.89327748]
 [ 4.95092102  4.05829386  4.98859597  1.05643814 -0.06065678]
 [ 4.17374486  4.79363348  4.0403715   0.80020294  1.2147315 ]
 [ 0.05517909  0.9344607   2.01282146  4.93654707  5.06819591]
 [ 0.91785348  2.09757003  0.98091237  4.09446399  4.89847501]]

[[5 5 4 1 1]
 [4 5 5 0 1]
 [5 4 5 1 0]
 [4 5 4 1 1]
 [0 1 2 5 5]
 [1 2 1 4 5]]


#### Task 4

In [87]:
# Function to predict ratings using iterative SVD
def iterative_predict_ratings(new_user_ratings, original_ratings, k, max_iter=10):
    # Initialize with the new user's ratings
    updated_ratings = np.vstack([original_ratings, new_user_ratings])
    
    # Iterative prediction process
    for iteration in range(max_iter):
        U, sigma, VT = np.linalg.svd(updated_ratings, full_matrices=False)
        Sigma = np.diag(sigma)

        U_reduced = U[:, :k]
        Sigma_reduced = Sigma[:k, :k]
        VT_reduced = VT[:k, :]

        predicted_ratings = np.dot(U_reduced, np.dot(Sigma_reduced, VT_reduced))
        
        updated_ratings[-1] = predicted_ratings[-1]

    final_predicted_ratings = updated_ratings[-1]  # Last row corresponds to the new user
    return final_predicted_ratings

# UserA's known ratings: M1 = 5, M4 = 1
userA_ratings = np.array([5, 3, 3, 1, 3])  # Initial guess for M2, M3, M5 as 3
userA_prediction = iterative_predict_ratings(userA_ratings, A, k=4)

print("\nFinal Predicted Ratings for UserA:\n", userA_prediction)

# UserB's known ratings: M2 = 1, M5 = 5
userB_ratings = np.array([3, 1, 3, 3, 5])  # Initial guess for M1, M3, M4 as 3
userB_prediction = iterative_predict_ratings(userB_ratings, A, k=4)

print("\nFinal Predicted Ratings for UserB:\n", userB_prediction)

# Determine preferences for UserA
action_movies_A = userA_prediction[:3]
comedy_movies_A = userA_prediction[3:]
userA_fan = "Action movies" if np.mean(action_movies_A) > np.mean(comedy_movies_A) else "Comedy movies"

# Determine preferences for UserB
action_movies_B = userB_prediction[:3]
comedy_movies_B = userB_prediction[3:]
userB_fan = "Action movies" if np.mean(action_movies_B) > np.mean(comedy_movies_B) else "Comedy movies"

print(f"\nUserA is a fan of: {userA_fan}")
print(f"UserB is a fan of: {userB_fan}")



Final Predicted Ratings for UserA:
 [4.84100722 3.94708469 2.17091143 2.29230152 1.82613927]

Final Predicted Ratings for UserB:
 [3.03461526 1.59832698 2.30940618 4.79840628 3.46423869]

UserA is a fan of: Action movies
UserB is a fan of: Comedy movies


### Question 2

##### 2a

In [88]:
# Function that computes the matrix-vector product A * p without storing A
def matrix_vector_product(p):
    n = len(p)
    result = np.zeros(n)
    
    for i in range(n):
        result[i] += 2 * (i + 1)**2 * p[i]  # for diagonal
        if i > 0:
            result[i] += (2 * (i + 2)**2 + i**2) * p[i - 1]  # Lower diagonal
        if i < n - 1:
            result[i] += ((i + 1)**2 + (i + 3)**2) * p[i + 1]  # Upper diagonal
    
    return result

# Conjugate Gradient Method
def conjugate_gradient(b, x0=None, tol=1e-8, max_iter=1000):
    n = 10
    if x0 is None:
        x0 = np.zeros(n)  # Initial guess
    
    r = b - matrix_vector_product(x0)           # Residual
    p = r.copy()
    x = x0.copy()
    
    rk_dot = np.dot(r, r)
    
    for i in range(max_iter):
        Ap = matrix_vector_product(p)
        alpha = rk_dot / np.dot(p, Ap)
        
        # Update the solution x
        x += alpha * p
        
        # Update residual
        r -= alpha * Ap
        
        rkp1_dot = np.dot(r, r)
        
        # Check for convergence
        if np.sqrt(rkp1_dot) < tol:
            break
        
        # Update the search direction
        beta = rkp1_dot / rk_dot
        p = r + beta * p
        
        rk_dot = rkp1_dot
    
    return x

# Vector b as defined in the problem
b = np.zeros(10)
for i in range(10):
    b[i] = (i + 1) / 2.0

# Solve using the Conjugate Gradient Method
x = conjugate_gradient(b)

# Print the solution
print("Solution vector x:\n", x)

Solution vector x:
 [ 1.25499487 -0.84498171 -3.12260739 -0.7245698   3.08815706 -0.34434029
 -2.39322812  2.25508162  1.69858822 -3.67720302]


##### 2b

In [89]:
# Matrix construction
A = np.zeros((10, 10))
for i in range(10):
    A[i][i] = 2 * (i + 1) ** 2      # for i = j
    if i > 0:
        A[i][i - 1] = 2 * (i + 2) ** 2 + (i) ** 2           # for i = j + 1
    if i < 9:
        A[i][i + 1] = (i + 1) ** 2 + (i + 3) ** 2           # for i + 1 = j 
# print(A)

# Vector b as defined in the problem
b = np.zeros(10)
for i in range(10):
    b[i] = (i + 1) / 2.0
# print(b)

# Solving using numpy
x = np.linalg.solve(A, b)
print("Solution vector x using numpy:\n", x)    


Solution vector x using numpy:
 [ 0.02618956  0.04476209  0.00721508 -0.00709725  0.03464274  0.01881651
 -0.02615845  0.02290478  0.04016843 -0.03987201]


##### 2c    

In [90]:
from scipy.sparse.linalg import cg

# Solve using SciPy's Conjugate Gradient Method
x_scipy, info = cg(A, b)
print("Solution vecotr x using scipy:\n", x_scipy)

# Check if the solver converged (info == 0 means successful convergence)
if info == 0:
    print("\nSciPy's Conjugate Gradient method converged successfully.")
else:
    print("\nSciPy's Conjugate Gradient method did not converge.")

Solution vecotr x using scipy:
 [-0.83568122  0.47028927  2.52366284  0.67933209 -0.14508061 -2.19994619
  1.67186345  0.31764905 -0.94163439  1.33557458]

SciPy's Conjugate Gradient method did not converge.


As can be clealy seen, all the three solutions are different with Conjugate Gradient Descent **not converging** to the optimal solution when using scipy. This is expected seeing that the matrix A is not symmetric. Below is the implementation of the three methods when A is symmetric.

In [91]:
# Function that computes the matrix-vector product A * p without storing A
def matrix_vector_product(p):
    n = len(p)
    result = np.zeros(n)
    
    for i in range(n):
        result[i] += 2 * (i + 1)**2 * p[i]  # for diagonal
        if i > 0:
            result[i] += ((i + 2)**2 + i**2) * p[i - 1]  # Lower diagonal
        if i < n - 1:
            result[i] += ((i + 1)**2 + (i + 3)**2) * p[i + 1]  # Upper diagonal
    
    return result

# Conjugate Gradient Method
def conjugate_gradient(b, x0=None, tol=1e-8, max_iter=1000):
    n = 10
    if x0 is None:
        x0 = np.zeros(n)  # Initial guess
    
    r = b - matrix_vector_product(x0)           # Residual
    p = r.copy()
    x = x0.copy()
    
    rk_dot = np.dot(r, r)
    
    for i in range(max_iter):
        Ap = matrix_vector_product(p)
        alpha = rk_dot / np.dot(p, Ap)
        
        # Update the solution x
        x += alpha * p
        
        # Update residual
        r -= alpha * Ap
        
        rkp1_dot = np.dot(r, r)
        
        # Check for convergence
        if np.sqrt(rkp1_dot) < tol:
            break
        
        # Update the search direction
        beta = rkp1_dot / rk_dot
        p = r + beta * p
        
        rk_dot = rkp1_dot
    
    return x

# Vector b as defined in the problem
b = np.zeros(10)
for i in range(10):
    b[i] = (i + 1) / 2.0

# Solve using the Conjugate Gradient Method
x = conjugate_gradient(b)

# Print the solution
print("Solution vector x:\n", x)

############################==============================############################
# Matrix construction
A = np.zeros((10, 10))
for i in range(10):
    A[i][i] = 2 * (i + 1) ** 2      # for i = j
    if i > 0:
        A[i][i - 1] = (i + 2) ** 2 + (i) ** 2           # for i = j + 1
    if i < 9:
        A[i][i + 1] = (i + 1) ** 2 + (i + 3) ** 2           # for i + 1 = j 
# print(A)

# Vector b as defined in the problem
b = np.zeros(10)
for i in range(10):
    b[i] = (i + 1) / 2.0
# print(b)

# Solving using numpy
x = np.linalg.solve(A, b)
print("Solution vector x using numpy:\n", x)  

############################==============================############################

from scipy.sparse.linalg import cg

# Solve using SciPy's Conjugate Gradient Method
x_scipy, info = cg(A, b)
print("Solution vector x using scipy:\n", x_scipy)  

# Check if the solver converged (info == 0 means successful convergence)
if info == 0:
    print("\nSciPy's Conjugate Gradient method converged successfully.")
else:
    print("\nSciPy's Conjugate Gradient method did not converge.")

Solution vector x:
 [-0.06295363  0.06259073  0.05644052 -0.0225807   0.01545394  0.03920946
 -0.00966672  0.00404918  0.02889256 -0.00418148]
Solution vector x using numpy:
 [-0.06295363  0.06259073  0.05644052 -0.0225807   0.01545394  0.03920946
 -0.00966672  0.00404918  0.02889256 -0.00418148]
Solution vector x using scipy:
 [-0.06295363  0.06259073  0.05644052 -0.0225807   0.01545394  0.03920946
 -0.00966672  0.00404918  0.02889256 -0.00418148]

SciPy's Conjugate Gradient method converged successfully.


Here, it is clear that the three methods yeild the same solution. 